In [1]:
# 틀린놈 다시 돌려서 검사 (?)

In [3]:
import pandas as pd
import numpy as np
import os
import json
from sklearn.model_selection import train_test_split

In [4]:
DATA_IN_PATH = './data_in/'
DATA_OUT_PATH = './data_out/'

TRAIN_Q1_DATA_FILE = 'train_q1.npy'
TRAIN_Q2_DATA_FILE = 'train_q2.npy'
TRAIN_LABEL_DATA_FILE = 'train_label.npy'

train_q1_data = np.load(open(DATA_IN_PATH + TRAIN_Q1_DATA_FILE, 'rb'))
train_q2_data = np.load(open(DATA_IN_PATH + TRAIN_Q2_DATA_FILE, 'rb'))
train_labels = np.load(open(DATA_IN_PATH + TRAIN_LABEL_DATA_FILE, 'rb'))

In [5]:
# 2개의 질문이 따로 떨어져있음
# numpy stack() 을 이용해서 하나로 합칠 것

train_input = np.stack((train_q1_data, train_q2_data), axis=1)
# numpy stack =>  2개의 넘파이 배열을 붙여주는건데,
# 축(axis)을 새로 생성해서 붙이는 것.  <- 기존거와는 다름

In [6]:
print(train_q1_data.shape, train_q2_data.shape)
# 이럼 2개

(298526, 31) (298526, 31)


In [7]:
# axis 1 으로 2개를 
print(train_input.shape)
# 2번째 축을 새로 생성해서 이런식으로 붙인 것

(298526, 2, 31)


In [8]:
print(train_input[0])

[[   4   13    5  338  377    8   18  202    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0]
 [   4    9    5  338  377   20 1841    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0]]


In [9]:
# axis를 주는 순간 , 축을 새로 생성해서 붙임 -> stack

In [10]:
# 학습 데이터 일부를 검증용으로 때어내자
train_input, eval_input, train_label, eval_label = \
        train_test_split(train_input, train_labels, test_size=0.2, random_state=4242)
                         
# 이렇게 학습은 끝남

In [ ]:
# XGBoost는 이렇게 설치
# conda install -c anaconda py-xgboost
# python으로 구현된 xgboost란 소리

In [12]:
import xgboost as xgb
# import 되면 제대로 설치됐다는 소리

In [14]:
# 변경
train_data = xgb.DMatrix(train_input.sum(axis=1), label = train_label)
                         # column 방향으로 sum을 한 것
eval_data = xgb.DMatrix(eval_input.sum(axis=1), label=eval_label)
# label을 항상 같이 주게 되어있음

data_list = [(train_data, 'train'), (eval_data, 'valid')]

In [15]:
print(data_list[0])

(<xgboost.core.DMatrix object at 0x0000020EF1B5DCA0>, 'train')


In [16]:
# 학습방법
# params로 dict를 만들고
params = {}
params['objective'] = 'binary:logistic'
params['eval_metric'] = 'rmse' # root mean square error를 고차함수로 쓰겠다는 소리

bst = xgb.train(params, train_data, num_boost_round=1000, evals=data_list,   # 검증할때 뭘 쓸겨냐
               early_stopping_rounds=10)  # 검사할때, 더 좋아지지 않으면 early stop을 하겠다는 소리

[0]	train-rmse:0.48376	valid-rmse:0.48406
[1]	train-rmse:0.47352	valid-rmse:0.47401
[2]	train-rmse:0.46683	valid-rmse:0.46739
[3]	train-rmse:0.46189	valid-rmse:0.46263
[4]	train-rmse:0.45832	valid-rmse:0.45929
[5]	train-rmse:0.45555	valid-rmse:0.45667
[6]	train-rmse:0.45350	valid-rmse:0.45487
[7]	train-rmse:0.45200	valid-rmse:0.45368
[8]	train-rmse:0.44953	valid-rmse:0.45128
[9]	train-rmse:0.44800	valid-rmse:0.44987
[10]	train-rmse:0.44691	valid-rmse:0.44895
[11]	train-rmse:0.44476	valid-rmse:0.44688
[12]	train-rmse:0.44397	valid-rmse:0.44626
[13]	train-rmse:0.44322	valid-rmse:0.44568
[14]	train-rmse:0.44208	valid-rmse:0.44474
[15]	train-rmse:0.44154	valid-rmse:0.44432
[16]	train-rmse:0.44114	valid-rmse:0.44398
[17]	train-rmse:0.43989	valid-rmse:0.44294
[18]	train-rmse:0.43913	valid-rmse:0.44240
[19]	train-rmse:0.43882	valid-rmse:0.44215
[20]	train-rmse:0.43849	valid-rmse:0.44191
[21]	train-rmse:0.43794	valid-rmse:0.44140
[22]	train-rmse:0.43753	valid-rmse:0.44118
[23]	train-rmse:0.436

In [17]:
# 멈췄으면, 캐글에 제출해야 결과를 알 수 있으니 -> 파일로 생성
TEST_Q1_DATA_FILE = 'test_q1.npy'
TEST_Q2_DATA_FILE = 'test_q2.npy'
TEST_ID_DATA_FILE = 'test_id.npy'

test_q1_data = np.load(open(DATA_IN_PATH + TEST_Q1_DATA_FILE, 'rb'))
test_q2_data = np.load(open(DATA_IN_PATH + TEST_Q2_DATA_FILE, 'rb'))
test_id_data = np.load(open(DATA_IN_PATH + TEST_ID_DATA_FILE, 'rb'), allow_pickle=True)

In [18]:
# 자 평가를 해보자
test_input = np.stack((test_q1_data, test_q2_data), axis=1)
test_data = xgb.DMatrix(test_input.sum(axis=1))
test_predict = bst.predict(test_data)

In [19]:
if not os.path.exists(DATA_OUT_PATH):
    os.makedirs(DATA_OUT_PATH)
    
output = pd.DataFrame({'test_id': test_id_data, 'is_duplicate': test_predict})
# 이렇게 나온 놈을
output.to_csv(DATA_OUT_PATH + 'simple_xgb.csv', index=False)

In [20]:
# 파일 나왔으면 캐글에 올려서 채점을 해보자

In [ ]:
# https://www.kaggle.com/c/quora-question-pairs/submit